In [2]:
from dateutil.parser import parse 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet as pr
#import mplcursors
from datetime import datetime

Matplotlib created a temporary config/cache directory at C:\Users\MATHEU~1\AppData\Local\Temp\matplotlib-hlvynpja because the default path (C:\Users\Matheus Alves\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
c:\Users\Matheus Alves\Documents\git_tcc\time_series_precipitation\ts\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
import warnings
warnings.filterwarnings("ignore")

# Carregando Base de Dados, Dividindo entre Teste e Treino e Alterando para Padrão Prophet


In [4]:
rainfall = pd.read_csv(r"C:\Users\Matheus Alves\Documents\git_tcc\time_series_precipitation\Dataset\dados_chuvas1.csv", parse_dates=True)

In [5]:
rainfall['Date'] = pd.to_datetime(rainfall['Date'], format="%Y/%m/%d")
#rainfall['Date'] = rainfall['Date'].dt.strftime('%Y-%m')

In [6]:
train = rainfall.iloc[:-14]
test = rainfall.iloc[-14:]

In [7]:
 train.columns = ['ds','y']

# Instanciando Prophet e Fazendo uma Previsão Básica

In [18]:
rainfall['month'] = pd.DatetimeIndex(rainfall.Date).month

rainfall_jan = rainfall[(rainfall.month == 1)]
rainfall_feb = rainfall[(rainfall.month == 2)]
rainfall_mar = rainfall[(rainfall.month == 3)]
rainfall_dec = rainfall[(rainfall.month == 12)]

all_dfs = [rainfall_dec,rainfall_jan,rainfall_feb,rainfall_mar]

rainfall_summer = pd.concat(all_dfs).reset_index(drop=True)
rainfall_summer

,Date,Precipitation,month
0,1984-12-31,139.6,12
1,1985-12-31,176.3,12
2,1986-12-31,383.2,12
3,1987-12-31,146.3,12
4,1988-12-31,279.6,12
...,...,...,...
148,2017-03-31,160.4,3
149,2018-03-31,220.5,3
150,2019-03-31,239.6,3
151,2020-03-31,69.4,3


In [20]:
#adicionando os meses de verão para uma previsão mais precisa
summer = pd.DataFrame({'holiday': 'summer',
                       'ds': rainfall_summer['Date']})

In [21]:
model = pr(interval_width=0.95, daily_seasonality=False, holidays=summer)
model.fit(train)
future = model.make_future_dataframe(periods=14, freq='M')

00:21:16 - cmdstanpy - INFO - Chain [1] start processing
00:21:16 - cmdstanpy - INFO - Chain [1] done processing


In [22]:
rainfall_pred = model.predict(future)

In [23]:
rainfall_pred

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,holidays,...,summer,summer_lower,summer_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1984-01-31,122.344130,148.980463,416.213592,122.344130,122.344130,163.269972,163.269972,163.269972,22.712846,...,22.712846,22.712846,22.712846,140.557126,140.557126,140.557126,0.0,0.0,0.0,285.614102
1,1984-02-29,122.362626,120.507348,386.169188,122.362626,122.362626,124.983940,124.983940,124.983940,22.712846,...,22.712846,22.712846,22.712846,102.271094,102.271094,102.271094,0.0,0.0,0.0,247.346566
2,1984-03-31,122.382397,39.410405,300.234328,122.382397,122.382397,59.355900,59.355900,59.355900,22.712846,...,22.712846,22.712846,22.712846,36.643054,36.643054,36.643054,0.0,0.0,0.0,181.738297
3,1984-04-30,122.401530,-40.504895,214.227740,122.401530,122.401530,-36.558427,-36.558427,-36.558427,0.000000,...,0.000000,0.000000,0.000000,-36.558427,-36.558427,-36.558427,0.0,0.0,0.0,85.843104
4,1984-05-31,122.421301,-49.091039,220.845658,122.421301,122.421301,-37.096576,-37.096576,-37.096576,0.000000,...,0.000000,0.000000,0.000000,-37.096576,-37.096576,-37.096576,0.0,0.0,0.0,85.324726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,2021-09-30,131.061372,-50.812095,218.226834,131.061167,131.061572,-46.010289,-46.010289,-46.010289,0.000000,...,0.000000,0.000000,0.000000,-46.010289,-46.010289,-46.010289,0.0,0.0,0.0,85.051084
453,2021-10-31,131.080753,-7.019981,257.951426,131.080516,131.080989,-8.420291,-8.420291,-8.420291,0.000000,...,0.000000,0.000000,0.000000,-8.420291,-8.420291,-8.420291,0.0,0.0,0.0,122.660462
454,2021-11-30,131.099508,12.957582,272.956675,131.099227,131.099776,9.726965,9.726965,9.726965,0.000000,...,0.000000,0.000000,0.000000,9.726965,9.726965,9.726965,0.0,0.0,0.0,140.826472
455,2021-12-31,131.118888,99.899445,362.726289,131.118584,131.119191,102.980275,102.980275,102.980275,22.712846,...,22.712846,22.712846,22.712846,80.267429,80.267429,80.267429,0.0,0.0,0.0,234.099162


In [24]:
previsao = rainfall_pred[['ds','yhat']].iloc[-14:]

In [25]:
test

,Date,Precipitation
443,2020-12-31,273.2
444,2021-01-31,215.5
445,2021-02-28,180.6
446,2021-03-31,138.4
447,2021-04-30,54.4
448,2021-05-31,44.4
449,2021-06-30,16.2
450,2021-07-31,39.4
451,2021-08-31,44.4
452,2021-09-30,39.5


In [26]:
previsao.columns = ['Date','Precipitation']
previsao.round(1)

,Date,Precipitation
443,2020-12-31,244.0
444,2021-01-31,301.0
445,2021-02-28,254.8
446,2021-03-31,210.4
447,2021-04-30,92.0
448,2021-05-31,84.1
449,2021-06-30,69.3
450,2021-07-31,46.4
451,2021-08-31,40.5
452,2021-09-30,85.1


# Avaliando o Modelo

In [27]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

In [28]:
real_value = np.array(test['Precipitation'])
pred_value = np.array(previsao['Precipitation'])

In [29]:
smape = MeanAbsolutePercentageError(symmetric=True)
smape(real_value, pred_value)

0.41574138884805306